# Crypto ML Model scaling the entire dataset

In [54]:
import json
import requests
import pandas as pd
import hvplot.pandas
import panel as pn
import holoviews as hv
import numpy as np
from pathlib import Path
%matplotlib inline

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [3]:
from sklearn.cluster import KMeans

In [4]:

# Coin Metrics API endpoint url
url = 'https://community-api.coinmetrics.io/v4/timeseries/asset-metrics?'

# Dates to get historical data for the last 3 years
start_date = pd.Timestamp("2021-01-01").isoformat()
end_date = pd.Timestamp("2021-10-05").isoformat()

# Number of records to retrieve
page_size = 10000

# Define the list of crypto assets to analyze - Unable to loop through CSV file
#crypto_asset = ['1inch', 'aave', 'ada', 'alpha', 'ant', 'bal', 'bat', 'bch']

crypto_asset = ['1inch', 'aave', 'ada', 'alpha', 'ant', 'bal', 'bat', 'bch', 'bnb', 'bnb_eth', 'bsv', 'btc', 'btg', 'busd', 'comp', 'cro', 'crv', 'cvc', 'dai', 'dash', 'dcr', 'dgb', 'dgx', 'doge', 'dot', 'drgn', 'elf', 'eng', 'eos_eth', 'etc', 'eth', 'ethos', 'ftt', 'fun', 'fxc', 'gas', 'gno', 'gnt', 'gusd', 'hbtc', 'hedg', 'ht', 'husd', 'knc', 'lend', 'leo_eth', 'link', 'loom', 'lsk', 'ltc', 'maid', 'mana', 'mco', 'mkr', 'neo', 'nxm', 'omg', 'pax', 'paxg', 'pay', 'poly', 'powr', 'ppt', 'qash', 'ren', 'renbtc', 'rep', 'sai', 'snt', 'snx', 'srm', 'sushi', 'swrv', 'trx_eth', 'tusd', 'uma', 'uni', 'usdc', 'usdk', 'usdt', 'usdt_eth', 'usdt_omni', 'usdt_trx', 'vtc', 'wbtc', 'weth', 'wnxm', 'wtc', 'xaut', 'xlm', 'xrp', 'xtz', 'xvg', 'yfi', 'zec', 'zrx'
               ]

def get_data(metric):
    
    # Save the JSON response as lists
    symbol = []
    time = []
    value = []

    # Loop through the selected tokens
    for token in crypto_asset:
        
        # Build URL with the corresponding token, metric, and dates
        get_url = f"{url}&assets={token}&metrics={metric}&start_time={start_date}&end_time={end_date}&page_size={page_size}"

        # Get the response from Coin Metrics API is JSON format
        response = requests.get(get_url).json()

        # Iterate through the response to add the relevant items to its corresponding list
        for item in response['data']:

            symbol.append(item['asset'])
            time.append(item['time'])
            value.append(item[metric])
            
    # Build the dataframe with the data
    data_df = pd.DataFrame(
    {'Token': symbol,
     'Date': time,
     metric: value
    })
    
    # Format the date for easy reading 
    data_df['Date'] = pd.to_datetime(data_df['Date'], format="%Y/%m/%d").dt.date
    
    # Set Date as the index
    data_df.index = data_df['Date']
    
    # Change the value type to 'float' to allow graphing
    data_df[metric] = data_df[metric].astype(float)
    
    # Define columns
    data_df = data_df.drop(columns='Date')
    data_df.columns = ['Token', metric]
    
    # Format the DataFrame to allow xgraphing
    #updated_data_df = data_df.pivot_table('Value', ['Date'], 'Token')
    
    #return updated_data_df
    
    return data_df
    

In [5]:
# Apply get_data and parse the data for all metrics
active_addresses = get_data('AdrActCnt')
transaction_count = get_data('TxCnt')
supply = get_data('SplyCur')
mc = get_data('CapMrktCurUSD')
mc_real = get_data('CapRealUSD')
velocity = get_data('VelCur1yr')

In [6]:
# Reset indexes and prepare to concatenate DataFrames
active_addresses = active_addresses.reset_index()
transaction_count = transaction_count.reset_index()
supply = supply.reset_index()
mc = mc.reset_index()
mc_real = mc_real.reset_index()
velocity = velocity.reset_index()

## KMeans using StandardScaler

In [63]:
# Create a new DataFrame with all fundamental data 
df = pd.concat([active_addresses, transaction_count, supply, mc, mc_real, velocity], join='outer', axis=1)

# Remove duplicate Token column
df = df.loc[:, ~df.columns.duplicated()]

# Rename columns
df = df.rename(columns={'AdrActCnt': 'Active Addresses', 'TxCnt': 'Transaction Count', 'SplyCur': 'Current Supply', 'CapMrktCurUSD': 'Market Cap', 'CapRealUSD': 'Real Market Cap', 'VelCur1yr': 'Velocity'})

# Set Date as index
df = df.set_index("Date")

# Replace NaN with zeros
df = df.fillna(0)

df

,Token,Active Addresses,Transaction Count,Current Supply,Market Cap,Real Market Cap,Velocity
Date,,,,,,,
2021-01-01,1inch,2251.0,4717.0,1.500000e+09,1.725317e+09,1.083266e+08,0.719532
2021-01-02,1inch,1739.0,3372.0,1.500000e+09,1.545368e+09,1.000618e+08,0.741131
2021-01-03,1inch,1380.0,2845.0,1.500000e+09,1.679718e+09,1.077241e+08,0.767223
2021-01-04,1inch,1149.0,2226.0,1.500000e+09,1.542135e+09,1.001020e+08,0.793863
2021-01-05,1inch,1878.0,3393.0,1.500000e+09,1.883339e+09,1.242016e+08,0.825911
...,...,...,...,...,...,...,...
2021-10-01,zrx,466.0,684.0,1.000000e+09,0.000000e+00,0.000000e+00,7.968792
2021-10-02,zrx,530.0,582.0,1.000000e+09,0.000000e+00,0.000000e+00,7.964344
2021-10-03,zrx,438.0,525.0,1.000000e+09,0.000000e+00,0.000000e+00,7.963604


In [72]:
df_act = df[df.Token.isin(['btc', 'eth', 'doge'])]
df_act = df_act.reset_index()
df_act.hvplot.hist('Active Addresses')

:Histogram   [Active Addresses]   (Active Addresses_count)

In [8]:
# Scaling the columns using StandardScaler
df_scaled_std = StandardScaler().fit_transform(df[['Active Addresses', 'Transaction Count', 'Current Supply', 'Market Cap', 'Real Market Cap', 'Velocity']])

# Creating a DataFrame with the scaled data
df_scaled_std = pd.DataFrame(df_scaled_std, columns=['Active Addresses', 'Transaction Count', 'Current Supply', 'Market Cap', 'Real Market Cap', 'Velocity'])

df_scaled_std

,Active Addresses,Transaction Count,Current Supply,Market Cap,Real Market Cap,Velocity
0,-0.253292,-0.206561,-0.243918,-0.174987,-0.233065,-0.685311
1,-0.257085,-0.209478,-0.243918,-0.176886,-0.233273,-0.684595
2,-0.259745,-0.210622,-0.243918,-0.175468,-0.233080,-0.683730
3,-0.261456,-0.211964,-0.243918,-0.176920,-0.233272,-0.682847
4,-0.256055,-0.209433,-0.243918,-0.173319,-0.232665,-0.681784
...,...,...,...,...,...,...
24721,-0.266516,-0.215309,-0.265465,-0.193196,-0.235795,-0.445001
24722,-0.266041,-0.215530,-0.265465,-0.193196,-0.235795,-0.445148
24723,-0.266723,-0.215654,-0.265465,-0.193196,-0.235795,-0.445173
24724,-0.266856,-0.215812,-0.265465,-0.193196,-0.235795,-0.445156


In [9]:
# Finding the best number for k using the Elbow method

# Create a list with the k values to try
k = list(range(1, 11))

# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia for k
for i in k:
    model = KMeans(n_clusters=i, random_state=0)
    model.fit(df_scaled_std)
    inertia.append(model.inertia_)

# Create a dict with the data to plot the elbow
elbow_data = {
    'k': k,
    'inertia': inertia
}

# Create a DataFrame with the elbow data
df_elbow = pd.DataFrame(elbow_data)

In [10]:
# Plot the elbow curve
df_elbow.hvplot.line(x='k', y='inertia', title='Elbow Curve', xticks=k)

:Curve   [k]   (inertia)

In [11]:
# Applying the KMeans algo

# Initilize the KMeans model using 5 clusters
model = KMeans(n_clusters=3)

# Fit the model
model.fit(df_scaled_std)

# Predict clusters
token_segments = model.predict(df_scaled_std)

In [12]:
# Create a new column in the scaled DF with the predicted clusters
df_scaled_std['Token Segment'] = token_segments

In [13]:
# Adding the Token column to the scaled df
df = df.reset_index()
df_scaled_std['Token'] = df['Token']

# Adding the Date column to the scaled df
df_scaled_std['Date'] = df['Date']
df_scaled_std

,Active Addresses,Transaction Count,Current Supply,Market Cap,Real Market Cap,Velocity,Token Segment,Token,Date
0,-0.253292,-0.206561,-0.243918,-0.174987,-0.233065,-0.685311,0,1inch,2021-01-01
1,-0.257085,-0.209478,-0.243918,-0.176886,-0.233273,-0.684595,0,1inch,2021-01-02
2,-0.259745,-0.210622,-0.243918,-0.175468,-0.233080,-0.683730,0,1inch,2021-01-03
3,-0.261456,-0.211964,-0.243918,-0.176920,-0.233272,-0.682847,0,1inch,2021-01-04
4,-0.256055,-0.209433,-0.243918,-0.173319,-0.232665,-0.681784,0,1inch,2021-01-05
...,...,...,...,...,...,...,...,...,...
24721,-0.266516,-0.215309,-0.265465,-0.193196,-0.235795,-0.445001,0,zrx,2021-10-01
24722,-0.266041,-0.215530,-0.265465,-0.193196,-0.235795,-0.445148,0,zrx,2021-10-02
24723,-0.266723,-0.215654,-0.265465,-0.193196,-0.235795,-0.445173,0,zrx,2021-10-03
24724,-0.266856,-0.215812,-0.265465,-0.193196,-0.235795,-0.445156,0,zrx,2021-10-04


In [14]:
# Plot the scatter plot
df_scaled_std.hvplot.scatter(x='Active Addresses', y='Transaction Count', by='Token Segment')

:NdOverlay   [Token Segment]
   :Scatter   [Active Addresses]   (Transaction Count)

### KMeans using MinMaxScaler

In [15]:
# Initialize the scaler
scaler = MinMaxScaler()

df_scaled_mm = scaler.fit_transform(df[['Active Addresses', 'Transaction Count', 'Current Supply', 'Market Cap', 'Real Market Cap', 'Velocity']])

# Creating a DataFrame with the scaled data
df_scaled_mm = pd.DataFrame(df_scaled_mm, columns=['Active Addresses', 'Transaction Count', 'Current Supply', 'Market Cap', 'Real Market Cap', 'Velocity'])
df_scaled_mm

,Active Addresses,Transaction Count,Current Supply,Market Cap,Real Market Cap,Velocity
0,0.001647,0.000575,0.011413,0.001456,0.000269,0.002783
1,0.001273,0.000411,0.011413,0.001304,0.000249,0.002886
2,0.001010,0.000347,0.011413,0.001417,0.000268,0.003011
3,0.000841,0.000271,0.011413,0.001301,0.000249,0.003138
4,0.001374,0.000414,0.011413,0.001589,0.000308,0.003291
...,...,...,...,...,...,...
24721,0.000341,0.000083,0.007608,0.000000,0.000000,0.037355
24722,0.000388,0.000071,0.007608,0.000000,0.000000,0.037334
24723,0.000321,0.000064,0.007608,0.000000,0.000000,0.037330
24724,0.000307,0.000055,0.007608,0.000000,0.000000,0.037333


In [16]:
# Finding the best number for k using the Elbow method

# Create a list with the k values to try
k = list(range(1, 11))

# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia for k
for i in k:
    model = KMeans(n_clusters=i, random_state=0)
    model.fit(df_scaled_mm)
    inertia.append(model.inertia_)

# Create a dict with the data to plot the elbow
elbow_data = {
    'k': k,
    'inertia': inertia
}

# Create a DataFrame with the elbow data
df_elbow = pd.DataFrame(elbow_data)

In [17]:
# Plot the elbow curve
df_elbow.hvplot.line(x='k', y='inertia', title='Elbow Curve', xticks=k)

:Curve   [k]   (inertia)

In [18]:
# Applying the KMeans algo

# Initilize the KMeans model using 5 clusters
model = KMeans(n_clusters=4)

# Fit the model
model.fit(df_scaled_mm)

# Predict clusters
token_segments = model.predict(df_scaled_mm)

In [19]:
# Create a new column in the scaled DF with the predicted clusters
df_scaled_mm['Token Segment'] = token_segments

In [48]:
# Adding the Token column to the scaled df
df_scaled_mm['Token'] = df['Token']

# Adding the Date column to the scaled df
df_scaled_mm['Date'] = df['Date']
df_scaled_mm

,Active Addresses,Transaction Count,Current Supply,Market Cap,Real Market Cap,Velocity,Token Segment,Token,Date
0,0.001647,0.000575,0.011413,0.001456,0.000269,0.002783,0,1inch,2021-01-01
1,0.001273,0.000411,0.011413,0.001304,0.000249,0.002886,0,1inch,2021-01-02
2,0.001010,0.000347,0.011413,0.001417,0.000268,0.003011,0,1inch,2021-01-03
3,0.000841,0.000271,0.011413,0.001301,0.000249,0.003138,0,1inch,2021-01-04
4,0.001374,0.000414,0.011413,0.001589,0.000308,0.003291,0,1inch,2021-01-05
...,...,...,...,...,...,...,...,...,...
24721,0.000341,0.000083,0.007608,0.000000,0.000000,0.037355,0,zrx,2021-10-01
24722,0.000388,0.000071,0.007608,0.000000,0.000000,0.037334,0,zrx,2021-10-02
24723,0.000321,0.000064,0.007608,0.000000,0.000000,0.037330,0,zrx,2021-10-03
24724,0.000307,0.000055,0.007608,0.000000,0.000000,0.037333,0,zrx,2021-10-04


In [ ]:
df_scaled_mm.corr

In [22]:
# Plot the scatter plot
df_scaled_mm.hvplot.scatter(x='Market Cap', y='Active Addresses', c='Token', by='Token Segment')

:NdOverlay   [Token Segment]
   :Scatter   [Market Cap]   (Active Addresses,Token)

In [45]:
cluster_0 = df_scaled_mm[df_scaled_mm['Token Segment'] == 0]

In [46]:
cluster_0.Token.unique()

array(['1inch', 'aave', 'alpha', 'ant', 'bal', 'bat', 'bch', 'bsv', 'btg',
       'busd', 'comp', 'crv', 'cvc', 'dash', 'dcr', 'dgb', 'dgx', 'dot',
       'drgn', 'elf', 'eng', 'etc', 'eth', 'ethos', 'ftt', 'fun', 'gno',
       'gnt', 'gusd', 'hbtc', 'hedg', 'ht', 'husd', 'knc', 'lend',
       'leo_eth', 'link', 'loom', 'lsk', 'ltc', 'maid', 'mana', 'mco',
       'mkr', 'neo', 'nxm', 'omg', 'pax', 'paxg', 'pay', 'poly', 'powr',
       'ppt', 'qash', 'ren', 'renbtc', 'rep', 'snt', 'snx', 'srm',
       'sushi', 'swrv', 'uma', 'uni', 'usdk', 'usdt', 'usdt_omni',
       'usdt_trx', 'vtc', 'wbtc', 'wnxm', 'wtc', 'xaut', 'xtz', 'xvg',
       'yfi', 'zrx'], dtype=object)

In [44]:
cluster_3.hvplot.scatter(x='Market Cap', y='Active Addresses', c='Token', by='Token Segment')

:NdOverlay   [Token Segment]
   :Scatter   [Market Cap]   (Active Addresses,Token)